# Calculation and clustering of word embeddings (Word2Vec)

In preparation, we import the necessary packages and modules and make a few settings:

In [1]:
import gensim
import pandas as pd
from gensim.models import Word2Vec
import logging
import warnings
import re
import xml.etree.ElementTree as et

warnings.simplefilter(action='ignore', category=FutureWarning)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## 1. Read the corpus

Now we read in the corpus. For demonstration purposes, we use a corpus containing the election programmes of the German Bundestagswahl 2021 (you can download the corpus [here](https://github.com/fussballlinguist/btw21/blob/main/btw21_sentence.vrt)). Word2Vec needs a list of lists, ideally of tokenised sentences. Here we have a part-of-speech-tagged corpus (TreeTagger) in an XML format, in which sentences are marked up by the \<s\>-tag. We first inspect the structure of the document and display the first 40 lines.

In [2]:
!head -40 /Users/simon/Korpora/Politik/BTW21/btw21_sentence.vrt

<corpus>
<text id="CDU" party="CDU" election="Bundestag" year="2021" source="https://online.fliphtml5.com/kxyi/eyjg/">
<h1>
<s>
Das	ART	die
Programm	NN	Programm
für	APPR	für
Stabilität	NN	Stabilität
und	KON	und
Erneuerung	NN	Erneuerung
.	$.	.
</s>
<s>
GEMEINSAM	ADJD	gemeinsam
FÜR	APPR	für
EIN	ART	eine
MODERNES	ADJA	modern
DEUTSCHLAND	NE	Deutschland
.	$.	.
</s>
</h1>
<h2>
<s>
Einleitung	NN	Einleitung
</s>
</h2>
<p>
<s>
Deutschland	NE	Deutschland
ist	VAFIN	sein
ein	ART	eine
starkes	ADJA	stark
Land	NN	Land
.	$.	.
</s>
<s>
Das	PDS	die
ist	VAFIN	sein
vor	APPR	vor
allem	PIS	alle


To read in the corpus, we use an XML parser, access the \<s> elements and read in the lemmas one by one.

In [3]:
tree = et.parse("/Users/simon/Korpora/Politik/BTW21/btw21_sentence.vrt")
root = tree.getroot()
sentences = root.findall(".//s")

docs = []

for sentence in sentences:
    tokens = []
    sent = sentence.text
    lines = sent.split("\n")
    for line in lines:
        m = re.search('\t(\S+)$', line)
        if m:
            tokens.append(m.group(1))
    docs.append(tokens)

We inspect the corpus and display the first four sentences.

In [4]:
docs[0:4]

[['die', 'Programm', 'für', 'Stabilität', 'und', 'Erneuerung', '.'],
 ['gemeinsam', 'für', 'eine', 'modern', 'Deutschland', '.'],
 ['Einleitung'],
 ['Deutschland', 'sein', 'eine', 'stark', 'Land', '.']]

For plain text corpora, I recommend the library NLTK with the method PlaintextCorpusReader(https://www.nltk.org/howto/corpus.html#plaintext-corpus-reader). All we have to do is specify a path to a folder containing at least one txt file. The corpus is read in and automatically segmented into sentences and tokenised (albeit rather roughly, i.e. not language-specific). The cell is commented out here.

In [5]:
#from nltk.corpus.reader import PlaintextCorpusReader

#corpus_root = "/Users/simon/Korpora/Wortschatz/txt"
#corpus = PlaintextCorpusReader(corpus_root, ".*\.txt")

#docs = corpus.sents()

#docs[-3:-1]

In [6]:
#docs1[-1000]

## 2. Training the model

Now we train the model. We first set a few parameters and then start the training for ten epochs.

In [7]:
model = gensim.models.Word2Vec (docs, vector_size=100, window=10, min_count=3, workers=10)
model.train(docs,total_examples=len(docs),epochs=10)

2022-03-09 15:26:13,447 : INFO : collecting all words and their counts
2022-03-09 15:26:13,448 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-09 15:26:13,488 : INFO : PROGRESS: at sentence #10000, processed 170919 words, keeping 16002 word types
2022-03-09 15:26:13,517 : INFO : collected 22945 word types from a corpus of 293649 raw words and 16973 sentences
2022-03-09 15:26:13,518 : INFO : Creating a fresh vocabulary
2022-03-09 15:26:13,558 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=3 retains 6773 unique words (29.518413597733712%% of original 22945, drops 16172)', 'datetime': '2022-03-09T15:26:13.547387', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.15.7-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-03-09 15:26:13,559 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=3 leaves 274270 word corpus (93.4006245551662%% of original 293649, drops 19379)'

2022-03-09 15:26:14,256 : WARNING : Effective 'alpha' higher than previous training cycles
2022-03-09 15:26:14,257 : INFO : Word2Vec lifecycle event {'msg': 'training model with 10 workers on 6773 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10 shrink_windows=True', 'datetime': '2022-03-09T15:26:14.257284', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.15.7-x86_64-i386-64bit', 'event': 'train'}
2022-03-09 15:26:14,353 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-03-09 15:26:14,359 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-03-09 15:26:14,361 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-03-09 15:26:14,369 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-03-09 15:26:14,371 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-03-09 15:26:14,373 : INFO : worker thread fini

2022-03-09 15:26:15,183 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-03-09 15:26:15,184 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-03-09 15:26:15,187 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-09 15:26:15,190 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-03-09 15:26:15,191 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-03-09 15:26:15,192 : INFO : EPOCH - 8 : training on 293649 raw words (179798 effective words) took 0.1s, 1701975 effective words/s
2022-03-09 15:26:15,288 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-03-09 15:26:15,293 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-03-09 15:26:15,294 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-03-09 15:26:15,299 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-03-09 15:26:15,306 : INFO : worker threa

(1801056, 2936490)

Now we can query the so-called Nearest Neighbours and build data-driven synsets.

In [8]:
nn = model.wv.most_similar(positive=["senken"], topn=10)
pd.DataFrame(nn, columns=['Lemma', 'Similarity'])

,Lemma,Similarity
0,Trassenpreise,0.908312
1,absenken,0.866041
2,steigern,0.862329
3,erhöhen,0.847885
4,anheben,0.846826
5,Inflationsausgleich,0.832479
6,zwölf,0.831356
7,deckeln,0.831158
8,Mittelbau,0.829897
9,zurücknehmen,0.820135


We can also query the similarity value of two given words.

In [9]:
model.wv.similarity('Steuer', 'Einnahme')

0.6709892

And we can pass a list of words to find which one doesn't go with the others.

In [10]:
model.wv.doesnt_match("Deutschland Frankreich Russland Geld".split())

'Geld'

If we want to reuse the model later, we have to save it. There is a compressed binary format or a more memory-intensive format, but we need this, if we want to visualise the model in the Tensorflor Embedding Projector.

In [11]:
#model.save("/Users/simon/word2vec/btw_binary.model")
model.wv.save_word2vec_format('/Users/simon/word2vec/btw.model')

2022-03-09 15:26:15,483 : INFO : storing 6773x100 projection weights into /Users/simon/word2vec/btw.model


To visualise the data, we first have to transform it with a build-in command line script:

In [12]:
!python -m gensim.scripts.word2vec2tensor -i /Users/simon/word2vec/btw21.model -o /Users/simon/word2vec/btw

2022-03-09 15:26:16,546 - word2vec2tensor - INFO - running /Users/simon/opt/anaconda3/lib/python3.8/site-packages/gensim/scripts/word2vec2tensor.py -i /Users/simon/word2vec/btw21.model -o /Users/simon/word2vec/btw
2022-03-09 15:26:16,546 - keyedvectors - INFO - loading projection weights from /Users/simon/word2vec/btw21.model
2022-03-09 15:26:16,894 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (6773, 100) matrix of type float32 from /Users/simon/word2vec/btw21.model', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-03-09T15:26:16.880525', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.15.7-x86_64-i386-64bit', 'event': 'load_word2vec_format'}
2022-03-09 15:26:17,302 - word2vec2tensor - INFO - 2D tensor file saved to /Users/simon/word2vec/btw_tensor.tsv
2022-03-09 15:26:17,302 - word2vec2tensor - INFO - Tensor metadata file saved to /Users/simon/word2vec/btw_metadata.tsv
2022-03-09 15:26:17,303 - word

Then we can open the [Embedding Projector](http://projector.tensorflow.org/) and simply load the files.

## 3. Clustering

The following clustering method is taken from: https://medium.com/ml2vec/using-word2vec-to-analyze-reddit-comments-28945d8cee57

We first load the necessary modules.

In [13]:
from sklearn.cluster import KMeans;
from sklearn.neighbors import KDTree;
import numpy as np;

To estimate how many clusters we need, it can be helpful to know the size of the vocabulary. Here, 400 clusters of 15 words each would cover almost the entire vocabulary.

In [14]:
len(model.wv.key_to_index)

6773

We first extract all vectors for all words from the model and store them as a numpy array. If we inspect the first entry, we see a 100-element vector with values updated after training the neural network model.

In [15]:
Z = model.wv.vectors
Z[0]

array([-0.44304603,  0.21300451,  0.60321605, -0.23796766, -0.5121961 ,
       -0.71748793,  0.14152496,  0.41390356, -1.1633451 ,  2.053871  ,
        0.35370275, -0.04281733,  0.38291058,  0.6855531 , -1.0318252 ,
        0.45673   , -0.05231862,  0.2722953 , -0.9079075 , -0.3053914 ,
        0.8198671 , -0.39138657,  0.7018809 , -0.18910633, -0.34803474,
        0.17097898, -0.26677993, -0.0400619 ,  0.18167266, -0.17606921,
        0.01727034, -0.24493016,  0.1281781 , -0.37114528, -0.6244334 ,
       -0.25462478, -0.02473817, -0.41811374,  0.714525  , -1.218948  ,
       -1.1021954 , -0.8519388 , -1.0437161 , -0.17690618,  0.01533971,
        0.11944128, -1.1326582 ,  0.5126999 ,  0.3038832 ,  1.3198425 ,
       -0.06481726, -0.1321497 , -0.04715572,  0.78729826,  0.5399731 ,
       -0.16169515,  0.73471075,  0.26752666, -1.0868489 , -0.28869626,
        0.00593494, -0.42668003, -0.6967169 , -1.0640528 ,  0.22891253,
        0.00979269, -0.02059126, -0.6433033 ,  0.36343956,  0.65

Now we define the clustering method according to the k-means++ method (it will simply look at differences between vectors and centers) and apply it to calculate 400 clusters. This may take a while. 

In [16]:
def clustering_on_wordvecs(word_vectors, num_clusters):
    # Initalize a k-means object and use it to extract centroids
    kmeans_clustering = KMeans(n_clusters = num_clusters, init='k-means++');
    idx = kmeans_clustering.fit_predict(word_vectors);
    
    return kmeans_clustering.cluster_centers_, idx;

centers, clusters = clustering_on_wordvecs(Z, 400);
centroid_map = dict(zip(model.wv.index_to_key, clusters));

Next, we get words in each cluster that are closest to the cluster center. To do this, we initialize a KDTree on the word vectors, and query it for the Top K words on each cluster center. Using the index-to-key-dictionary, we than correspond each word vector back to it's original word representation and add them to a dataframe for easier printing.

In [17]:
def get_top_words(index_to_key, k, centers, wordvecs):
    tree = KDTree(wordvecs);

#Closest points for each Cluster center is used to query the closest 20 points to it.
    closest_points = [tree.query(np.reshape(x, (1, -1)), k=k) for x in centers];
    closest_words_idxs = [x[1] for x in closest_points];

#Word Index is queried for each position in the above array, and added to a Dictionary.
    closest_words = {};
    for i in range(0, len(closest_words_idxs)):
        closest_words['Cluster #' + str(i)] = [index_to_key[j] for j in closest_words_idxs[i][0]]

#A DataFrame is generated from the dictionary.
    df = pd.DataFrame(closest_words);
    df.index = df.index+1
    return df;

Now we query the top 15 for each cluster (this also takes a while).

In [18]:
top_words = get_top_words(model.wv.index_to_key, 15, centers, Z)

In order to be able to display all clusters here in the notebook, we still have to make a corresponding setting for pandas. Then we print the data frame.

In [19]:
pd.set_option('display.max_columns', None)
top_words

,Cluster #0,Cluster #1,Cluster #2,Cluster #3,Cluster #4,Cluster #5,Cluster #6,Cluster #7,Cluster #8,Cluster #9,Cluster #10,Cluster #11,Cluster #12,Cluster #13,Cluster #14,Cluster #15,Cluster #16,Cluster #17,Cluster #18,Cluster #19,Cluster #20,Cluster #21,Cluster #22,Cluster #23,Cluster #24,Cluster #25,Cluster #26,Cluster #27,Cluster #28,Cluster #29,Cluster #30,Cluster #31,Cluster #32,Cluster #33,Cluster #34,Cluster #35,Cluster #36,Cluster #37,Cluster #38,Cluster #39,Cluster #40,Cluster #41,Cluster #42,Cluster #43,Cluster #44,Cluster #45,Cluster #46,Cluster #47,Cluster #48,Cluster #49,Cluster #50,Cluster #51,Cluster #52,Cluster #53,Cluster #54,Cluster #55,Cluster #56,Cluster #57,Cluster #58,Cluster #59,Cluster #60,Cluster #61,Cluster #62,Cluster #63,Cluster #64,Cluster #65,Cluster #66,Cluster #67,Cluster #68,Cluster #69,Cluster #70,Cluster #71,Cluster #72,Cluster #73,Cluster #74,Cluster #75,Cluster #76,Cluster #77,Cluster #78,Cluster #79,Cluster #80,Cluster #81,Cluster #82,Cluster #83,Cluster #84,Cluster #85,Cluster #86,Cluster #87,Cluster #88,Cluster #89,Cluster #90,Cluster #91,Cluster #92,Cluster #93,Cluster #94,Cluster #95,Cluster #96,Cluster #97,Cluster #98,Cluster #99,Cluster #100,Cluster #101,Cluster #102,Cluster #103,Cluster #104,Cluster #105,Cluster #106,Cluster #107,Cluster #108,Cluster #109,Cluster #110,Cluster #111,Cluster #112,Cluster #113,Cluster #114,Cluster #115,Cluster #116,Cluster #117,Cluster #118,Cluster #119,Cluster #120,Cluster #121,Cluster #122,Cluster #123,Cluster #124,Cluster #125,Cluster #126,Cluster #127,Cluster #128,Cluster #129,Cluster #130,Cluster #131,Cluster #132,Cluster #133,Cluster #134,Cluster #135,Cluster #136,Cluster #137,Cluster #138,Cluster #139,Cluster #140,Cluster #141,Cluster #142,Cluster #143,Cluster #144,Cluster #145,Cluster #146,Cluster #147,Cluster #148,Cluster #149,Cluster #150,Cluster #151,Cluster #152,Cluster #153,Cluster #154,Cluster #155,Cluster #156,Cluster #157,Cluster #158,Cluster #159,Cluster #160,Cluster #161,Cluster #162,Cluster #163,Cluster #164,Cluster #165,Cluster #166,Cluster #167,Cluster #168,Cluster #169,Cluster #170,Cluster #171,Cluster #172,Cluster #173,Cluster #174,Cluster #175,Cluster #176,Cluster #177,Cluster #178,Cluster #179,Cluster #180,Cluster #181,Cluster #182,Cluster #183,Cluster #184,Cluster #185,Cluster #186,Cluster #187,Cluster #188,Cluster #189,Cluster #190,Cluster #191,Cluster #192,Cluster #193,Cluster #194,Cluster #195,Cluster #196,Cluster #197,Cluster #198,Cluster #199,Cluster #200,Cluster #201,Cluster #202,Cluster #203,Cluster #204,Cluster #205,Cluster #206,Cluster #207,Cluster #208,Cluster #209,Cluster #210,Cluster #211,Cluster #212,Cluster #213,Cluster #214,Cluster #215,Cluster #216,Cluster #217,Cluster #218,Cluster #219,Cluster #220,Cluster #221,Cluster #222,Cluster #223,Cluster #224,Cluster #225,Cluster #226,Cluster #227,Cluster #228,Cluster #229,Cluster #230,Cluster #231,Cluster #232,Cluster #233,Cluster #234,Cluster #235,Cluster #236,Cluster #237,Cluster #238,Cluster #239,Cluster #240,Cluster #241,Cluster #242,Cluster #243,Cluster #244,Cluster #245,Cluster #246,Cluster #247,Cluster #248,Cluster #249,Cluster #250,Cluster #251,Cluster #252,Cluster #253,Cluster #254,Cluster #255,Cluster #256,Cluster #257,Cluster #258,Cluster #259,Cluster #260,Cluster #261,Cluster #262,Cluster #263,Cluster #264,Cluster #265,Cluster #266,Cluster #267,Cluster #268,Cluster #269,Cluster #270,Cluster #271,Cluster #272,Cluster #273,Cluster #274,Cluster #275,Cluster #276,Cluster #277,Cluster #278,Cluster #279,Cluster #280,Cluster #281,Cluster #282,Cluster #283,Cluster #284,Cluster #285,Cluster #286,Cluster #287,Cluster #288,Cluster #289,Cluster #290,Cluster #291,Cluster #292,Cluster #293,Cluster #294,Cluster #295,Cluster #296,Cluster #297,Cluster #298,Cluster #299,Cluster #300,Cluster #301,Cluster #302,Cluster #303,Cluster #304,Cluster #305,Cluster #306,Cluster #307,Cluster #308,Cluster #309,Cluster #310,Cluster #311,Cluster #312,Cluster #313,Cluster #314,Cluster #315,C

We can save the cluster table.

In [21]:
top_words.to_csv('/Users/Simon/word2vec/btw_clusters.csv')